이 코드는 한국어 텍스트 데이터를 기반으로 여러 종류의 평가 지표(회귀 및 분류)를 예측하기 위한 딥러닝 모델을 구축하고, 훈련시키며, 평가하는 데 사용됩니다. 구체적인 내용은 다음과 같습니다:

1. **모델 정의**: 
   - `KoBERTRubricScorer`: 한국어 BERT(`KoBERT`) 모델을 사용하여 회귀 작업을 수행하는 모델입니다. 여러 회귀 작업(점수)을 예측합니다.
   - `KoBERTRubricClassifier`: `KoBERT`를 사용하여 분류 작업을 수행하는 모델입니다. 다양한 클래스를 분류합니다.
   - `KoBERTRubricMTLScorer`: 다중 작업 학습(Multi-Task Learning)을 위한 모델로, 회귀 및 분류 작업을 동시에 수행합니다.
   - `KoBERTRubricMTLRegressor`: 다중 회귀 작업을 수행하는 모델로, 여러 회귀 출력을 동시에 예측합니다.

2. **데이터셋 및 데이터 로더**:
   - `RubricScoringDataset`: 텍스트 데이터셋을 전처리하고, 토크나이징하여 모델이 사용할 수 있는 형식으로 변환합니다. 필요한 경우 데이터 증강을 수행합니다.
   - `create_data_loader`: 데이터셋을 배치(batch) 단위로 로드하고, 훈련 및 검증/테스트를 위해 셔플 옵션을 관리합니다.

3. **훈련 및 검증 로직**:
   - `RubricScorerTrainer`: 모델을 훈련시키고 검증하는 클래스입니다. 각 에폭 동안 훈련 손실과 정확도를 계산하고, 검증 데이터셋에 대해 성능을 평가합니다.

4. **앙상블**:
   - `RubricScorerEnsemble`: 훈련된 여러 모델의 예측 결과를 앙상블하여 최종 예측 결과를 생성합니다. 여기서는 단순 평균(average) 방식을 사용합니다.

5. **메인 함수 (`main`)**:
   - 데이터셋을 로드하고 전처리합니다.
   - 정의된 모델을 초기화하고 훈련시킵니다.
   - 훈련된 모델을 앙상블하여 테스트 데이터에 대한 예측을 수행합니다.
   - 평가 함수를 사용하여 모델 성능을 측정하고 결과를 출력합니다.

전체적으로 이 코드는 딥러닝 기반의 다중 작업 학습 모델을 구축하고 평가하기 위한 완전한 파이프라인을 제공합니다. 데이터 전처리부터 모델 훈련, 앙상블, 평가에 이르기까지 전체적인 딥러닝 작업 흐름을 다룹니다.

```json
[
    {
        "prompt": "인공지능의 발전이 사회에 미치는 영향에 대해 논하시오.",
        "rubric": [
            {"name": "내용의 적절성", "description": "주제와 관련된 내용을 적절히 다루었는가?", "max_score": 5},
            {"name": "논리적 구조", "description": "글의 구조가 논리적으로 잘 조직되었는가?", "max_score": 5},
            {"name": "어휘의 다양성", "description": "다양하고 적절한 어휘를 사용하였는가?", "max_score": 3},
            {"name": "문법 및 맞춤법", "description": "문법 및 맞춤법이 정확한가?", "max_score": 2}
        ],
        "response": "인공지능의 발전은 사회에 많은 변화를 가져올 것입니다. ...",
        "scores": [3, 4, 2, 1]
    },
    ...
]
```

1. **KoBERTRubricScorer**:
   - 회귀 작업을 위해 설계되었으며, 주어진 프롬프트와 응답에 대한 숫자 점수를 예측하는 것이 목표입니다.
   - KoBERT 모델을 사용하여 입력 텍스트를 인코딩하고, 풀링된 출력을 선형 회귀기를 통해 `num_tasks`에 대한 점수를 예측합니다.
   - 각 평가 측면에 대한 연속적인 점수를 필요로 하는 시나리오에 적합합니다.

2. **KoBERTRubricClassifier**:
   - 분류 작업을 위해 설계되었으며, 텍스트를 사전에 정의된 클래스로 분류하는 것이 목표입니다.
   - 구조는 `KoBERTRubricScorer`와 유사하지만, 끝에 회귀기 대신 분류기 레이어를 사용합니다.
   - `num_classes` 매개변수는 분류 작업에 대한 카테고리 또는 클래스의 수를 지정합니다.
   - 응답을 우수, 좋음, 보통, 나쁨과 같은 범주로 평가하는 데 적합합니다.

3. **KoBERTRubricMTLScorer (다중 작업 학습기)**:
   - 회귀와 분류 작업을 결합한 다중 작업 모델입니다.
   - 회귀 작업을 위한 별도의 회귀 헤드와 텍스트 분류를 위한 분류 헤드를 사용합니다.
   - 점수 매기기와 분류 모두가 필요한 시나리오에 적합하며, 두 작업에 대한 공유 표현을 활용합니다.

요약하면:
- `KoBERTRubricScorer`와 `KoBERTRubricMTLRegressor`는 회귀 기반 점수 매기기에 사용되며, 후자는 여러 회귀 작업을 독립적으로 처리합니다.
- `KoBERTRubricClassifier`는 분류 작업에 특화되어 있습니다.
- `KoBERTRubricMTLScorer`는 점수 매기기와 분류를 동시에 처리할 수 있어, 복잡한 루브릭 기반 평가 시스템에 다재다능하게 활용될 수 있습니다.

`KoBERTRubricMTLScorer` 모델은 다중 작업 학습(Multi-Task Learning, MTL)을 위해 설계된 모델로서, 텍스트 기반 평가에서 회귀와 분류 작업을 동시에 수행할 수 있습니다. 이 모델의 원리와 구성 요소를 세부적으로 설명하겠습니다:

### 모델 아키텍처

- **KoBERT 레이어**: 
  - 이 모델은 한국어 BERT 모델인 KoBERT를 기반으로 합니다. KoBERT는 텍스트 데이터를 입력으로 받아 문맥을 고려한 특징 벡터로 변환합니다. 
  - 입력 텍스트는 토크나이즈되어 `input_ids`, `attention_mask`, `token_type_ids` 등의 형태로 변환되며, 이를 모델에 전달합니다.

- **출력 처리**:
  - KoBERT 모델을 통해 생성된 출력은 풀링된 출력(`pooled_output`)을 사용합니다. 이는 통상적으로 문서의 전반적인 의미를 담은 벡터로, BERT 모델의 첫 번째 토큰([CLS] 토큰)에 해당하는 출력입니다.

- **회귀 헤드**: 
  - 회귀 작업을 위해 여러 개의 회귀 헤드를 사용합니다. 각 회귀 헤드는 `pooled_output`을 입력으로 받아, 선형 변환을 통해 각각의 작업에 대한 예측값(점수)을 생성합니다. 이를 위해 `nn.Linear` 레이어를 사용하며, 각 회귀 작업마다 하나씩 있습니다.

- **분류 헤드**: 
  - 분류 작업을 수행하기 위한 별도의 선형 레이어를 포함합니다. 이 레이어는 마찬가지로 `pooled_output`을 입력으로 받아, 지정된 클래스 수(`num_classes`)에 대한 예측을 수행합니다.

### 원리

- 입력 텍스트는 KoBERT를 통해 문맥적 특징을 반영한 벡터로 인코딩됩니다.
- 인코딩된 벡터는 dropout 레이어를 통과하여 과적합을 방지합니다.
- 회귀 헤드는 각각의 루브릭 요소나 평가 기준에 대한 점수를 예측합니다.
- 분류 헤드는 입력 텍스트가 속하는 범주를 예측합니다.
- 이를 통해 한 번의 순방향 패스(forward pass)로 여러 작업의 예측을 동시에 수행할 수 있으며, 관련 작업 간에 유익한 정보를 공유할 수 있습니다.

### 활용

`KoBERTRubricMTLScorer`는 한 번의 모델 실행으로 여러 평가 기준에 대한 점수를 동시에 예측할 수 있기 때문에, 효율성과 성능이 뛰어난 텍스트 평가 시스템을 구축하는 데 이상적입니다. 특히 교육적 평가, 콘텐츠 분석, 감성 분석 등 다양한 분야에서 활용될 수 있습니다.

분류 헤드는 주어진 텍스트를 분석하여 특정 범주(클래스)에 할당하는 역할을 수행합니다. 예를 들어, 학생이 작성한 에세이를 평가하는 상황에서 분류 헤드는 에세이의 질을 분류할 수 있습니다.

### 에세이 평가 예시:

#### 입력 데이터:
- 학생이 작성한 에세이 텍스트가 입력으로 주어집니다.
- 에세이의 내용은 "인공지능의 발전이 사회에 미치는 영향"에 대해 논하는 내용을 담고 있습니다.

#### 분류 헤드의 역할:
- 분류 헤드는 에세이의 품질을 평가하기 위해 다음과 같은 범주로 분류할 수 있습니다:
  - 우수 (Excellent)
  - 양호 (Good)
  - 보통 (Average)
  - 미흡 (Poor)

#### 처리 과정:
1. **텍스트 인코딩**: 에세이 텍스트는 KoBERT 모델을 통해 인코딩되어, 문맥을 고려한 특징 벡터로 변환됩니다.
2. **특징 분석**: 변환된 특징 벡터는 분류 헤드에 입력되어, 에세이가 각 범주에 속할 확률을 계산합니다.
3. **결과 예측**: 확률이 가장 높은 범주가 에세이의 최종 평가 등급으로 선택됩니다.

### 예시 활용:
분류 헤드를 사용하여 학교 교사나 평가자가 학생의 에세이를 효율적으로 평가할 수 있게 도와줍니다. 이 시스템은 대량의 에세이를 빠르게 분류하고, 교육적 피드백 제공에 필요한 정보를 얻는 데 유용합니다.

이러한 방식으로 분류 헤드는 텍스트 데이터의 내용을 기반으로 정해진 분류 체계에 따라 분류하는 중요한 역할을 수행합니다.

회귀 헤드는 주어진 텍스트 데이터에 대해 연속적인 값이나 점수를 예측하는 역할을 수행합니다. 이는 분류와는 다르게 구체적인 수치를 출력하여, 텍스트의 양적 특성을 평가하는 데 사용됩니다.

### 회귀 헤드의 역할:

#### 예시: 에세이 점수화
학생의 에세이에 대해 구체적인 평가 지표(예: 내용의 적절성, 논리적 구조, 어휘 사용 등)에 따른 점수를 부여하는 경우, 회귀 헤드는 각 지표에 대한 숫자 점수를 예측합니다.

#### 처리 과정:
1. **텍스트 인코딩**: 에세이 텍스트는 먼저 KoBERT 모델을 통해 문맥적 특징을 고려한 벡터로 변환됩니다.
2. **점수 예측**: 변환된 벡터는 회귀 헤드로 전달되며, 각 평가 지표에 대한 점수를 예측하기 위한 선형 변환을 수행합니다.
3. **결과 출력**: 각 평가 지표에 대한 예측된 점수가 출력되며, 이는 실제 수치로 해석됩니다.

### 활용:
- **교육 분야**: 학생의 에세이나 보고서를 평가할 때 각 항목에 대한 세부적인 점수를 부여하여 교육적 피드백을 제공합니다.
- **리서치 분석**: 연구 논문이나 보고서의 품질, 중요성 등을 평가하는 데 사용될 수 있습니다.
- **제품 리뷰**: 고객의 리뷰를 분석하여 제품의 만족도, 품질 등을 수치화할 수 있습니다.

회귀 헤드는 이처럼 텍스트 데이터에 대한 구체적이고 정량적인 분석을 제공하며, 다양한 분야에서 텍스트의 질적 특성을 수치로 변환하는 데 중요한 역할을 합니다.

주어진 데이터셋에 대해 모델의 각 부분은 다음과 같은 역할을 수행합니다:

1. **KoBERT 레이어**:
   - **역할**: 텍스트의 문맥적 이해를 가능하게 하여, 주어진 "prompt"와 "response"를 효과적으로 인코딩합니다.
   - **작동 방식**: Transformer 기반의 BERT 모델로서, 입력된 텍스트를 고차원 특징 벡터로 변환합니다. 이는 텍스트의 의미를 포착하여 각 단어의 문맥적 관계를 이해하는 데 중요합니다.

2. **회귀 헤드**:
   - **역할**: 에세이나 텍스트의 각 루브릭 항목(예: "내용의 적절성", "논리적 구조" 등)에 대해 구체적인 점수(회귀 값)를 예측합니다.
   - **작동 방식**: KoBERT에서 추출된 특징 벡터를 사용하여, 각 루브릭 항목에 대한 연속적인 점수를 예측하는 선형 레이어가 여기에 해당합니다.

3. **분류 헤드**:
   - **역할**: 텍스트가 특정 범주(예: "문법 및 맞춤법"의 점수가 낮음, 중간, 높음 등)에 속하는지를 분류합니다.
   - **작동 방식**: 문법과 맞춤법과 같은 특정 평가 기준을 분류하는 데 사용되며, KoBERT 출력을 기반으로 해당 범주에 속할 확률을 계산합니다.

4. **KoBERTRubricMTLScorer**:
   - **역할**: 다중 작업 학습을 통해, 회귀 및 분류 작업을 동시에 수행합니다. 이는 동일한 입력에서 여러 타입의 출력을 예측하는 데 사용됩니다.
   - **작동 방식**: 텍스트 입력에 대해 여러 회귀 점수와 분류 결과를 동시에 출력할 수 있습니다. 각각의 출력은 특정 루브릭 평가 항목을 대상으로 합니다.

각 부분은 데이터셋의 'prompt', 'response', 그리고 'rubric' 항목에서 제공되는 정보를 처리하고, 이를 바탕으로 해당 텍스트의 평가 기준에 따른 점수를 예측합니다. 회귀 헤드는 각 루브릭 요소에 대한 수치 점수를 제공하고, 분류 헤드는 더 일반적인 평가(예: 좋음, 보통, 나쁨)를 제공합니다. KoBERTRubricMTLScorer는 이 두 가지 타입의 예측을 통합하여, 보다 포괄적인 텍스트 평가를 수행합니다.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer, BertConfig
from torch.utils.data import DataLoader
from konlpy.tag import Okt
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
import random
from sklearn.model_selection import train_test_split

In [ ]:
# 모델 정의
class KoBERTRubricScorer(nn.Module):
    def __init__(self, num_tasks, hidden_size=768, dropout_rate=0.1):
        super().__init__()
        config = BertConfig.from_pretrained('monologg/kobert')
        config.num_labels = num_tasks
        self.bert = BertModel.from_pretrained('monologg/kobert', config=config)
        self.dropout = nn.Dropout(dropout_rate)
        self.regressor = nn.Linear(hidden_size, num_tasks)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        return logits

class KoBERTRubricClassifier(nn.Module):
    def __init__(self, num_classes, hidden_size=768, dropout_rate=0.1):
        super().__init__()
        config = BertConfig.from_pretrained('monologg/kobert')
        config.num_labels = num_classes
        self.bert = BertModel.from_pretrained('monologg/kobert', config=config)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

class KoBERTRubricMTLScorer(nn.Module):
    def __init__(self, num_tasks, num_classes, hidden_size=768, dropout_rate=0.1, criteria_descriptions=None):
        super().__init__()
        config = BertConfig.from_pretrained('monologg/kobert')
        self.bert = BertModel.from_pretrained('monologg/kobert', config=config)
        self.dropout = nn.Dropout(dropout_rate)
        
        if criteria_descriptions is not None:
            self.criteria_embeddings = self.embed_criteria(criteria_descriptions)
            concat_size = hidden_size * 2 + self.criteria_embeddings.size(1)  # Update the size to account for concatenation
        else:
            self.criteria_embeddings = None
            concat_size = hidden_size  # No concatenation with criteria embeddings

        self.regression_heads = nn.Linear(concat_size, num_tasks)
        self.classification_head = nn.Linear(concat_size, num_classes)

    def embed_criteria(self, criteria_descriptions):
        criteria_tokens = self.bert.tokenizer(criteria_descriptions, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            criteria_embeddings = self.bert.embeddings(criteria_tokens['input_ids'])
            criteria_embeddings = criteria_embeddings.mean(dim=1)  # Average across token embeddings
        return criteria_embeddings

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs.last_hidden_state
        pooled_output = outputs.pooler_output
        
        if self.criteria_embeddings is not None:
            attention_scores = torch.matmul(sequence_output, self.criteria_embeddings.transpose(0, 1))
            attention_weights = torch.softmax(attention_scores, dim=-1)
            context_vector = torch.matmul(attention_weights.transpose(1, 2), sequence_output)
            context_vector = context_vector.mean(dim=1)  # Average across criteria
            pooled_output = torch.cat((pooled_output, context_vector), dim=-1)  # Concatenate pooled output and context vector
        
        pooled_output = self.dropout(pooled_output)
        regression_outputs = self.regression_heads(pooled_output)
        classification_output = self.classification_head(pooled_output)
        return regression_outputs, classification_output

In [ ]:
# 데이터셋 정의
class RubricScoringDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length, okt=None, stopwords=None, synonym_dict=None, augment_ratio=0.0):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.okt = okt
        self.stopwords = stopwords
        self.synonym_dict = synonym_dict
        self.augment_ratio = augment_ratio

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        prompt = ''
        response = str(item['response'])  # Ensure response is a string
        rubric_text = ' '.join([f"{item['rubric1']}", f"{item['rubric2']}", f"{item['rubric3']}"])
        text = f"{prompt} {response} {rubric_text}"
        
        if self.okt is not None and self.stopwords is not None:
            tokens = self.okt.morphs(response)  # Morphological analysis
            tokens = [token for token in tokens if token not in self.stopwords]
            response = ' '.join(tokens)

        if self.augment_ratio > 0 and random.random() < self.augment_ratio:
            response = self.augment_text(response)

        text = f"{prompt} {response} {rubric_text}"
        inputs = self.tokenizer(text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        regression_labels = torch.tensor(item[['score1', 'score2', 'score3']].astype(float).values, dtype=torch.float32)
        
        # classification_label = torch.tensor(item['score4'], dtype=torch.long)
        classification_label = torch.tensor(int(round(item[['score1', 'score2', 'score3']].astype(float).mean())), dtype=torch.long)
        return inputs['input_ids'].squeeze(), inputs['attention_mask'].squeeze(), regression_labels, classification_label

    def augment_text(self, text):
        if self.synonym_dict is not None:
            words = text.split()
            for i, word in enumerate(words):
                if word in self.synonym_dict and random.random() < 0.1:
                    words[i] = random.choice(self.synonym_dict[word])
            text = ' '.join(words)
        return text

In [ ]:
# 훈련 및 검증 트레이너
class RubricScorerTrainer:
    def __init__(self, model, train_dataloader, val_dataloader, optimizer, regression_criterion, classification_criterion, device):
        self.model = model
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.optimizer = optimizer
        self.regression_criterion = regression_criterion
        self.classification_criterion = classification_criterion
        self.device = device

    def train(self, num_epochs):
        self.model.to(self.device)
        for epoch in range(num_epochs):
            self.model.train()
            train_loss = 0.0
            for batch in self.train_dataloader:
                input_ids, attention_mask, regression_labels, classification_labels = [data.to(self.device) for data in batch]
                self.optimizer.zero_grad()
                
                outputs = self.model(input_ids, attention_mask)

                # Check if the model returns one or two outputs
                if isinstance(outputs, tuple):  # Handles case with multiple outputs
                    regression_outputs, classification_outputs = outputs
                    regression_loss = self.regression_criterion(regression_outputs, regression_labels)
                    classification_loss = self.classification_criterion(classification_outputs, classification_labels)
                    loss = regression_loss + classification_loss
                else:  # Handles single output (regression only)
                    regression_loss = self.regression_criterion(outputs, regression_labels)
                    loss = regression_loss

                loss.backward()
                self.optimizer.step()
                train_loss += loss.item()

            train_loss /= len(self.train_dataloader)
            val_regression_loss, val_classification_loss, val_accuracy, val_f1 = self.validate()
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Regression Loss: {val_regression_loss:.4f}, Val Classification Loss: {val_classification_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}")


    def validate(self):
        self.model.eval()
        regression_loss = 0.0
        classification_loss = 0.0  # Initialize even if not used
        total_loss = 0.0
        classification_preds = []
        classification_labels = []

        with torch.no_grad():
            for batch in self.val_dataloader:
                input_ids, attention_mask, regression_labels, classification_labels_batch = [data.to(self.device) for data in batch]
                outputs = self.model(input_ids, attention_mask)

                if isinstance(outputs, tuple):  # Handles case with multiple outputs
                    regression_outputs, classification_outputs = outputs
                    regression_loss += self.regression_criterion(regression_outputs, regression_labels).item()
                    classification_loss += self.classification_criterion(classification_outputs, classification_labels_batch).item()
                    classification_preds.append(classification_outputs.argmax(dim=1).cpu().numpy())
                    classification_labels.append(classification_labels_batch.cpu().numpy())
                else:  # Handles single output (regression only)
                    regression_loss += self.regression_criterion(outputs, regression_labels).item()

                # Handle total loss for logging if necessary
                total_loss = regression_loss + classification_loss

        num_batches = len(self.val_dataloader)
        regression_loss /= num_batches
        classification_loss /= num_batches if num_batches > 0 else 1  # Avoid division by zero

        # Calculate metrics if classification was performed
        if classification_preds:
            classification_preds = np.concatenate(classification_preds)
            classification_labels = np.concatenate(classification_labels)
            accuracy = accuracy_score(classification_labels, classification_preds)
            f1 = f1_score(classification_labels, classification_preds, average='weighted')
        else:
            accuracy = f1 = 0  # Default values if no classification

        return regression_loss, classification_loss, accuracy, f1

In [ ]:
# 앙상블 클래스 정의
class RubricScorerEnsemble:
    def __init__(self, models, device):
        self.models = models
        self.device = device

    def predict(self, dataloader):
        regression_predictions_list = []
        classification_predictions_list = []

        with torch.no_grad():
            for model in self.models.values():
                model.to(self.device)
                model.eval()
                regression_predictions = []
                classification_predictions = []

                for batch in dataloader:
                    input_ids, attention_mask, _, _ = [data.to(self.device) for data in batch]
                    outputs = model(input_ids, attention_mask)

                    if isinstance(outputs, tuple):
                        regression_output, classification_output = outputs
                        classification_predictions.append(classification_output.cpu().numpy())
                    else:
                        regression_output = outputs

                    regression_predictions.append(regression_output.cpu().numpy())

                # Aggregate across batches per model
                if regression_predictions:
                    regression_predictions_list.append(np.concatenate(regression_predictions, axis=0))
                if classification_predictions:
                    classification_predictions_list.append(np.concatenate(classification_predictions, axis=0))

        # Aggregate across models
        if regression_predictions_list:
            ensemble_regression_predictions = np.mean(regression_predictions_list, axis=0)
        else:
            ensemble_regression_predictions = None  # or appropriate default/fallback

        if classification_predictions_list:
            ensemble_classification_predictions = np.mean(classification_predictions_list, axis=0)
        else:
            ensemble_classification_predictions = None  # or appropriate default/fallback

        return ensemble_regression_predictions, ensemble_classification_predictions

In [ ]:
def create_data_loader(dataset, batch_size, shuffle=True):
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

def evaluate_regression(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return mse

def evaluate_classification(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, f1

In [ ]:
def main():
    data = pd.read_excel('dataset.xlsx')
    # 학습 데이터와 테스트 데이터로 분할
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    # 학습 데이터를 다시 학습 데이터와 검증 데이터로 분할
    train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

    # 토크나이저 및 전처리 도구 로드
    tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
    okt = Okt()
    stopwords = ['은', '는', '이', '가', '을', '를']
    synonym_dict = {'좋아하다': ['즐기다', '기뻐하다'], '훌륭하다': ['우수하다', '뛰어나다']}

    # 데이터셋 생성
    train_dataset = RubricScoringDataset(train_data, tokenizer, max_length=128, okt=okt, stopwords=stopwords, synonym_dict=synonym_dict, augment_ratio=0.1)
    val_dataset = RubricScoringDataset(val_data, tokenizer, max_length=128, okt=okt, stopwords=stopwords)
    test_dataset = RubricScoringDataset(test_data, tokenizer, max_length=128, okt=okt, stopwords=stopwords)

    # 데이터 로더 생성
    batch_size = 16
    train_dataloader = create_data_loader(train_dataset, batch_size, shuffle=True)
    val_dataloader = create_data_loader(val_dataset, batch_size, shuffle=False)
    test_dataloader = create_data_loader(test_dataset, batch_size, shuffle=False)

    # 디바이스 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 모델 생성 및 훈련
    models = {
        "Rubric Scorer": KoBERTRubricScorer(num_tasks=3),
        # "Rubric Classifier": KoBERTRubricClassifier(num_classes=5),
        "MTL Rubric Scorer": KoBERTRubricMTLScorer(num_tasks=3, num_classes=5)
    }

    trained_models = {}
    num_epochs = 1
    for name, model in models.items():
        regression_criterion = nn.MSELoss()
        classification_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
        trainer = RubricScorerTrainer(model, train_dataloader, val_dataloader, optimizer, regression_criterion, classification_criterion, device)
        print(f"Training {name}...")
        trainer.train(num_epochs)
        trained_models[name] = model

        # 훈련된 모델 가중치 저장
        model_path = f"{name}_weights.pt"
        torch.save(model.state_dict(), model_path)
        print(f"Saved {name} weights to {model_path}")

    # 앙상블 설정 및 예측
    ensemble = RubricScorerEnsemble(trained_models, device)
    ensemble_regression_predictions, ensemble_classification_predictions = ensemble.predict(test_dataloader)

    # 평가 및 결과 출력
    regression_labels = test_data[['score1', 'score2', 'score3']].values
    regression_mse = evaluate_regression(regression_labels, ensemble_regression_predictions)
    print(f"Ensemble Regression MSE: {regression_mse:.4f}")

    for (idx, item) in test_data.iterrows():
        test_data.loc[idx, 'score4'] = int(round(item[['score1', 'score2', 'score3']].astype(float).mean()))

    classification_labels = test_data['score4'].values
    classification_predictions = np.argmax(ensemble_classification_predictions, axis=1)
    accuracy, f1 = evaluate_classification(classification_labels, classification_predictions)
    print(f"Ensemble Classification Accuracy: {accuracy:.4f}")
    print(f"Ensemble Classification F1 Score: {f1:.4f}")

if __name__ == "__main__":
    main()